# Optimizing "Serial" Performance

At the heart of fast parallel code must be fast serial code. Parallelism can make a good serial code faster. But it can also make a bad code even worse. One can write terribly slow code in any language, including Julia. In this notebook we want to understand what makes Julia code slow and how to detect and avoid common pitfalls. This will lead to multiple concrete performance tips that will help you speed up your Julia code and to write more efficient code in the first place.

By far the most common reasons for slow Julia code are

* **too many (unnecessary) allocations**
* **break-down of type inference** (e.g. type instabilities)

## Avoid Unnecessary Allocations

Dynamic heap allocations are costly compared to floating point operations. Avoid them, in particular in "hot" loops, because they may trigger garbage collection.

In [ ]:
using BenchmarkTools

In [ ]:
@btime 1.2 + 3.4;
@btime Vector{Float64}(undef, 1);

### Example 1: Element-Wise Operations

In [ ]:
function f()
    x = [1, 2, 3]
    for i in 1:10_000
        x = x + 2 * x
    end
    return x
end

In [ ]:
@btime f();

* Huge number of allocations!
* Bad sign that they scale with the number of iterations!

#### Fix 1: Write explicit loops

In [ ]:
function f()
    x = [1, 2, 3]
    for i in 1:100_000
        for k in eachindex(x)
            x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end

@btime f();

#### Fix 2: Broadcasting (aka "More Dots")

(Recommendation: Old but great [blog post](https://julialang.org/blog/2017/01/moredots) by Steven G. Johnson ([related notebook](https://github.com/JuliaLang/www.julialang.org/blob/master/blog/_posts/moredots/More-Dots.ipynb)))

In [ ]:
function f()
    x = [1, 2, 3]
    for i in 1:100_000
        x = x .+ 2 .* x
    end
    return x
end

@btime f();

In [ ]:
function f()
    x = [1, 2, 3]
    for i in 1:100_000
        x .= x .+ 2 .* x
        # or put @. in front
    end
    return x
end

@btime f();

#### Fix 3: Immutable Datatypes (if possible)

In [ ]:
using StaticArrays

function f()
  x = @SVector [1, 2, 3]
  for i in 1:100_000
    x = x + 2 * x
  end
  return x
end

@btime f();

No dynamic heap allocations at all!

### Example 2: Linear Algebra

<details>
<summary>Spoilers</summary>
There's a good chance that there will not be a large performance difference between these.

That's likely because this is an intense enough benchmark that you may begin to run into CPU throttling with a large number of samples, and the slowdown caused by that throttling dominates the average time far more than the improvement of the code.

By lowering the sample size to on the order of dozens (or less) you can see that the first function (on my PC, this will vary) takes roughly 160ms, and the second takes 11ms, an order of magnitude faster.
    
A slight hint of this may be seen on the benchmark histogram: both have a small clump of results on the far left ast very low times, and the 'range' section of the benchmark has a much lower value for the imporved function.
</details>

In [ ]:
function f()
    A = rand(100, 100)
    B = rand(100, 100)
    s = 0.0
    for i in 1:1_000
        C = A * B
        s += C[i]
    end
    return A
end

a = @benchmark f()

#### Fix: Preallocate and reuse memory + in-place matrix-multipy

In [ ]:
using LinearAlgebra

function f()
    A = rand(100, 100)
    B = rand(100, 100)
    C = zeros(100, 100) # preallocate
    # C = Array{Float64, 2}(undef, 100, 100)  # A tad faster
    s = 0.0
    for i in 1:100
        mul!(C, A, B) # reuse / in-place matmul
        s += C[i]
    end
    return A
end

b = @benchmark f()

### Example 3: Array slicing

By default, array-slicing creates copies!

In [ ]:
using BenchmarkTools

X = rand(3, 3);

In [ ]:
f(Y) = Y[:, 1] .+ Y[:, 2] .+ Y[:, 3]

@btime f($X);

#### Fix: Views

In [ ]:
f(Y) = @views Y[:, 1] .+ Y[:, 2] .+ Y[:, 3]

# expands to
# f(Y) = view(Y, 1:3, 1) .+ view(Y, 1:3, 2) .+ view(Y, 1:3, 3)

@btime f($X);

(Note that [copying data isn't always bad](https://docs.julialang.org/en/v1/manual/performance-tips/#Copying-data-is-not-always-bad))

### Example 4: Vectorized Style

Let's say we want the sum of the sin of the numbers 1 to 10:

In [ ]:
@btime sum(sin.([k for k in 1:10]))  # Dot for broadcast notation
# sum(map(sin, [k for k in 1:10]));  # Equivalent to this

#### Fix: Call Directly on Variable

No need to call sin after the list is generated:

In [ ]:
@btime sum([sin(k) for k in 1:10]);

#### Fix: Generators and Laziness

In [ ]:
@btime sum(sin(k) for k in 1:10); # generator

In [ ]:
@btime sum(sin, k for k in 1:10); # two-argument version of sum

How about we do our own sum?

In [ ]:
@btime begin
    iter = map(sin, [k for k in 1:10])
    res = 0
    for i in iter
        res += i
    end
end

In [ ]:
@btime begin
    iter = Iterators.map(sin, [k for k in 1:10])  # Lazy map, on an vector
    res = 0
    for i in iter
        res += i
    end
end

In [ ]:
@btime begin
    iter = Iterators.map(sin, k for k in 1:10)  # Lazy map, on a generator
    res = 0
    for i in iter
        res += i
    end
end

In [ ]:
@btime begin
    iter = (sin(k) for k in 1:10)  # Only generator, already lazy
    res = 0
    for i in iter
        res += i
    end
end

This highlights the point from the first session: **user code is as performant as any other code**

Try doing this in Numpy and it's always orders of magnitude slower than the Julia version, with the user-implemented sum being twice as slow as the built-in Numpy sum.

## Avoid Type Instabilities

**Type stability**: A function `f` is type stable if for a given set of input argument types the return type is always the same.

In particular, it means that the type of the output of `f` cannot vary depending on the **values** of the inputs.

**Type instability**: The return type of a function `f` is not predictable just from the type of the input arguments alone.

Instructive example: `f(x) = rand() > 0.5 ? 1.23 : "string"`

### Example: Global scope

A typical cause of type instability are global variables.

From a compiler perspective, variables defined in global scope **can change their value and even their type(!) any time**.

In [ ]:
a = 2.0
b = 3.0

f() = 2 * a + b

In [ ]:
f()

In [ ]:
@code_warntype f()

In [ ]:
@code_llvm f()

#### Fix 1: Work in local scope

In [ ]:
function local_scope()
    a = 2.0
    b = 3.0

    f() = 2 * a + b

    return f()
end

local_scope()

In [ ]:
@code_warntype local_scope()

In [ ]:
@code_llvm local_scope()

In [ ]:
@code_native local_scope()

This is fast.

In fact, it's not just fast, but **as fast as it can be**! Julia has figured out the result of the calculation at compile-time and returns just the literal, i.e. `local_scope() = 7`.

#### Fix 2: Make globals `const`ant

In [ ]:
const C = 2.0
const D = 3.0

f() = 2 * C + D

f()

In [ ]:
@code_llvm f()

In [ ]:
@code_warntype f()

#### Fix 3: Write self-contained functions

In [ ]:
f(a, b) = 2 * a + b

In [ ]:
@code_llvm debuginfo=:none f(2.0, 3.0)

**Write functions not scripts!**

### Example: Multiple Returns

In [ ]:
function f(x, flag)
    if flag
        return convert(Vector{Float64}, x)
    else
        return convert(Vector{Int64}, round.(x))
    end
end

In [ ]:
@code_warntype f(rand(10), true)

In [ ]:
typeof(f(rand(10), true))

In [ ]:
typeof(f(rand(10), false))

#### Fix: Hint Return Type

In [ ]:
function f(x, flag)::Vector{Float64}
    if flag
        return 1:length(x)
    else
        return x
    end
end

In [ ]:
@code_warntype f(rand(10), true)

NB: Really, a `Union` return with a few types in it is not that bad as Julia typically handles this situation via union splitting. However it can make the compilers job harder.

## Avoid Abstract Field Types

A common reason for type inference to break are not-concretely typed fields in `struct`s

### Example 1

In [ ]:
using BenchmarkTools

In [ ]:
struct MyType
    x::Number
    y::String
end

f(a::MyType) = a.x^2 + sqrt(a.x)

In [ ]:
a = MyType(3.0, "test")

@code_warntype f(a);

In [ ]:
@btime f($a);

In [ ]:
@code_llvm f(a);

In [ ]:
typeof(a)

**"Type stability"**: A function `f` is type stable if for a given set of input argument types the return type is always the same and *concrete*.

#### Fix 1: Concrete typing

In [ ]:
struct MyTypeConcrete
    x::Float64
    y::String
end

f(b::MyTypeConcrete) = b.x^2 + sqrt(b.x)

In [ ]:
b = MyTypeConcrete(3.0, "test")
@code_warntype f(b)

In [ ]:
@btime f($b);

In [ ]:
@code_llvm f(b)

#### Fix 2: Type parameters

But what if I want to accept any kind of, say, `Number` and `AbstractString` for our type?

In [ ]:
struct MyTypeParametric{A <: Number, B <: AbstractString}
    x::A
    y::B
end

f(c::MyTypeParametric) = c.x^2 + sqrt(c.x)

In [ ]:
c = MyTypeParametric(3.0, "test")

In [ ]:
@code_warntype f(c)

From the type alone the compiler knows what the structure contains and can produce optimal code:

In [ ]:
@btime f($c);

In [ ]:
c = MyTypeParametric(Float32(3.0), SubString("test"))

In [ ]:
@btime f($c);

### Example 2 - Cooler Diagonal Matrix

Yesterday we created a diagonal matrix which could take in a `Vector` and create an object that acted like a diagonal matrix.

By replacing the `Vector` with an `AbstractVector`, we made this work with anything vector-like, like ranges.

However, this ended up being slower than the original implementation:

In [ ]:
struct DiagonalMatrix{T} <: AbstractArray{T,2}
    diag::Vector{T}
end

Base.size(D::DiagonalMatrix) = (length(D.diag), length(D.diag))

function Base.getindex(D::DiagonalMatrix, i::Int, j::Int)
    if i == j
        return D.diag[i]
    else
        return zero(eltype(D))
    end
end

Base.:+(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag + Db.diag)

In [ ]:
struct CoolerDiagonalMatrix{T} <: AbstractArray{T, 2}
    diag::AbstractVector{T}
end

Base.size(D::CoolerDiagonalMatrix) = (length(D.diag), length(D.diag))

function Base.getindex(D::CoolerDiagonalMatrix, i::Int, j::Int)
    if i == j
        return D.diag[i]
    else
        return zero(eltype(D))
    end
end

Base.:+(Da::CoolerDiagonalMatrix, Db::CoolerDiagonalMatrix) = CoolerDiagonalMatrix(Da.diag + Db.diag)

In [ ]:
DM = DiagonalMatrix(collect(1:100))
@btime $DM + $DM

DM_cool = CoolerDiagonalMatrix(collect(1:100))
@btime $DM_cool + $DM_cool;

In [ ]:
@code_warntype DM + DM

In [ ]:
@code_warntype DM_cool + DM_cool

This is the same as the previous problem: an `AbstractVector{Int64}` is not a concrete type, it means:

In [ ]:
subtypes(AbstractVector{Int64})

So really it's a union of all of the above types, not a single type!

We can avoid this with some slightly fancier code:

In [ ]:
struct MuchCoolerDiagonalMatrix{T, U <: AbstractVector} <: AbstractArray{T, 2}
    diag::U
end

Base.size(D::MuchCoolerDiagonalMatrix) = (length(D.diag), length(D.diag))

function Base.getindex(D::MuchCoolerDiagonalMatrix, i::Int, j::Int)
    if i == j
        return D.diag[i]
    else
        return zero(eltype(D))
    end
end

Base.:+(Da::MuchCoolerDiagonalMatrix, Db::MuchCoolerDiagonalMatrix) = MuchCoolerDiagonalMatrix(Da.diag + Db.diag)

In [ ]:
MuchCoolerDiagonalMatrix(1:10)

In [ ]:
typeof(1:10)

In [ ]:
MuchCoolerDiagonalMatrix{Int64, UnitRange{Int64}}(1:10)

In [ ]:
MuchCoolerDiagonalMatrix(diag::AbstractVector) = MuchCoolerDiagonalMatrix{eltype(diag), typeof(diag)}(diag)

In [ ]:
MuchCoolerDiagonalMatrix{Int64, UnitRange{Int64}}(1:10)

In [ ]:
DM = DiagonalMatrix(collect(1:100))
@btime $DM + $DM

DM_cool = CoolerDiagonalMatrix(collect(1:100))
@btime $DM_cool + $DM_cool;

DM_mcool = MuchCoolerDiagonalMatrix(collect(1:100))
@btime $DM_mcool + $DM_mcool;

In [ ]:
DM = DiagonalMatrix(collect(1:100))
@btime $DM + $DM

DM_cool = CoolerDiagonalMatrix(1:100)
@btime $DM_cool + $DM_cool;

DM_mcool = MuchCoolerDiagonalMatrix(1:100)
@btime $DM_mcool + $DM_mcool;

In [ ]:
@code_warntype DM_mcool + DM_mcool

## Avoid Untyped Containers

### Example

In [ ]:
function f()
    numbers = []
    for i in 1:10
        push!(numbers, i)
    end
    sum(numbers)
end

@btime f();

In [ ]:
@code_warntype f()

In [ ]:
typeof([])

In [ ]:
function f()
    numbers = Int[]
    for i in 1:10
        push!(numbers, i)
    end
    sum(numbers)
end

@btime f();

In [ ]:
@code_warntype f()

## Avoid Changing Variable Types

Variables in a function should not change type.

### Example

In [ ]:
function f()
    x = 1
    for i = 1:10
        x /= rand()
    end
    return x
end

@code_warntype f();

(On a side note: since the type can only vary between `Float64` and `Int64`, Julia can still produce reasonable code by *union splitting*. I recommend reading [this blog post](https://julialang.org/blog/2018/08/union-splitting) by Tim Holy.)

#### Fix 1: Initialize with correct type

In [ ]:
function f()
    x = 1.0
    for i = 1:10
        x /= rand()
    end
    return x
end

@code_warntype f()

#### Fix 2: Specify types (to get errors or to heal the problem by conversion)

In [ ]:
function f()
    x::Float64 = 1 # implicit conversion
    for i = 1:10
        x /= rand()
    end
    return x
end

@code_warntype f()

#### Fix 3: Special-case first iteration

In [ ]:
function f()
    x = 1 / rand()
    for i = 2:10
        x /= rand()
    end
    return x
end

@code_warntype f()

## Isolate Unavoidable Type Instabilities

Type instabilities can occur very naturally, for example when reading unknown user files or user input. Hence, not every instability can be avoided.

If that's the case, isolate your expensive computation from the instability by putting it in a separate *kernel function* (also known as introducing a [function barrier](https://docs.julialang.org/en/v1/manual/performance-tips/#kernel-functions)).

In [ ]:
data = Union{Int64,Float64,String}[4, 2.0, "test", 3.2, 1]

In [ ]:
function computation(data)
    x = 1.0
    for i in 1:100
        x = sin(data[1])
        x += data[2]
        x *= data[4]
    end
    return x
end

@code_warntype computation(data)

In [ ]:
@btime computation($data);

In [ ]:
function computation(data)
    a = data[1]
    b = data[2]
    c = data[4]
    return _computation_kernel(a, b, c)
end

function _computation_kernel(a, b, c)
    x = 1.0
    for i in 1:100
        x = sin(a)
        x += b
        x *= c
    end
    return x
end

In [ ]:
@code_warntype computation(data)

In [ ]:
@code_warntype _computation_kernel(data[1], data[2], data[4])

In [ ]:
@btime computation($data);

Note that the computational kernel function is fully type inferred.

## Access Arrays in Column-Major Order

<img src="../../static/column-major-2D.png" width=800px>

Excellent page on this topic: https://book.sciml.ai/notes/02-Optimizing_Serial_Code/

**Fastest varying loop index goes first.**

In [ ]:
function fcol!(M)
    for col in 1:size(M, 2)
        for row in 1:size(M, 1)
            M[row, col] = 42
        end
    end
    nothing
end

function frow!(M)
    for row in 1:size(M, 1)
        for col in 1:size(M, 2)
            M[row, col] = 42
        end
    end
    nothing
end

In [ ]:
@benchmark fcol!(A) setup=(A=Array{Int64, 2}(undef, 1000, 1000))

In [ ]:
@benchmark frow!(A) setup=(A=Array{Int64, 2}(undef, 1000, 1000))

Lots of cache misses for `frow`!

## Performance Annotations

### `@inbounds`

Disables bounds checks. (Julia may segfault if you use it wrongly!)

In [ ]:
function f()
    x = [1, 2, 3]
    for i in 1:100_000
        for k in 1:3
            x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end

@btime f();

In [ ]:
function f_inbounds()
    x = [1, 2, 3]
    for i in 1:100_000
        for k in 1:3
            @inbounds x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end

@btime f_inbounds();

### `@simd`

Enables SIMD optimizations that are potentially *unsafe*. Julia may execute loop iterations in arbitrary or overlapping order.

In [ ]:
x = rand(1000);

In [ ]:
function f(x)
    s = zero(eltype(x))
    for i in x
        s += i
    end
    return s
end

@btime f($x);

In [ ]:
function f_simd(x)
    s = zero(eltype(x))
    @simd for i in x
        s += i
    end
    return s
end

@btime f_simd($x);

(For integer input both versions have the same speed because integer addition is associative, in contrast to floating point arithmetics.)

In [ ]:
using CpuId
cpuinfo()

In [ ]:
x = rand(Float32, 1000);

@btime f($x);
@btime f_simd($x);

### `@fastmath`

Enables lots of floating point optimizations that are potentially *unsafe*! It trades accuracy for speed, so, [Beware of fast-math](https://simonbyrne.github.io/notes/fastmath/). (See the [LLVM Language Reference Manual](https://llvm.org/docs/LangRef.html#fast-math-flags) for more information on which compiler options it sets.)

There is `julia --math-mode=fast` to enable fast math globally.

Harmless example: **FMA - Fused Multiply Add**

In [ ]:
@noinline f(a, b, c) = a * b + c

@code_native debuginfo=:none f(1.0, 2.0, 3.0)

<img src="../../static/skylake_microarchitecture.png" width=700px>

**Source:** [Intel® 64 and IA-32 Architectures Optimization Reference Manual](https://software.intel.com/sites/default/files/managed/9e/bc/64-ia-32-architectures-optimization-manual.pdf)

In [ ]:
f_fma(a, b, c) = fma(a, b, c)

@code_native debuginfo=:none f_fma(1.0, 2.0, 3.0)

In [ ]:
f_fastmath(a, b, c) = @fastmath a * b + c

@code_native debuginfo=:none f_fastmath(1.0, 2.0, 3.0)

Benchmarking this probably won's show a difference due to the noise on the system dominating the tiny benefits of saving one instruction.

We get into this later

(P.S. In this specific case, [MuladdMacro.jl](https://github.com/SciML/MuladdMacro.jl) is a *safe* alternative.

## CPU operations vary in cost

http://ithare.com/infographics-operation-costs-in-cpu-clock-cycles/

### Example: Division vs multiplication

In [ ]:
div(x) = x ./ 1000
mul(x) = x .* 1e-3

In [ ]:
x = rand(1000)

@btime $x ./ 1000;
@btime $x .* 1e-3;

## Analysis Tools

### [Traceur.jl](https://github.com/MikeInnes/Traceur.jl)

**Basic automatic performance trap checker**. Essentially a codified version of the [performance tips](https://docs.julialang.org/en/v1/manual/performance-tips/) in the Julia documentation.

Important macro: [`@trace`](http://traceur.junolab.org/latest/#Traceur.@trace)

In [ ]:
using Traceur

a = 2.0
b = 3.0

f() = 2 * a + b

@trace f()

### [JET.jl](https://github.com/aviatesk/JET.jl)

**Static** code analyzer. (Doesn't execute the code!)

Important macros:
* `@report_opt`: check for potential optimization problems ([optimization analysis](https://aviatesk.github.io/JET.jl/stable/optanalysis/))
* `@report_call`: check for potential (general) errors ([error analysis](https://aviatesk.github.io/JET.jl/stable/jetanalysis/))

In [ ]:
using JET

a = 2.0
b = 3.0

f() = 2 * a + b

@report_opt f() # check for possible optimization problems

In [ ]:
f() = x + 2

@report_call f() # check for possible errors

In [ ]:
@report_opt f()

In [ ]:
@report_call sum("Hamburg")

### [Cthulhu.jl](https://github.com/JuliaDebug/Cthulhu.jl)

**Interactive code explorer** that let's you navigate through a nested function call-tree and apply macros like `@code_*`, or `@which`, and more. For example, one can recursively apply `@code_warntype` at different levels to detect the origin of a type instability. (Note though that it might take some time to master Cthulhu.)

Important macro: `@descend` (or directly `@descend_code_warntype`)

(Cthulhu isn't a debugger! It has only "static" information.)

In [ ]:
using Cthulhu

A = rand(10, 10)
B = rand(10, 10)

# @descend A*B # doesn't work in Jupyter -> use REPL

# Summary

- Start with optimising serial performance, bad serial code will be bad parallel code
- **Reduce number of allocations**
  - Use `@btime` to check the number of allocations
  - Avoid unnecessary allocations, huge no. of them is a bad sign

- Avoid temporary allocations
  - e.g. dot syntax for broadcasting is very useful
  - Pre-allocating result arrays can save time

- Slicing by default **copies data**, use `@views` or `view(A, 1:2, 1)` to get a non-copy view
- (Lazy) generators can be much faster for certain tasks

- **Type instability is VERY bad**
  - Use `@code_warntype` to check for type instability
  - Avoid (non-constant) globals or work only in local scope
  - Hint return types if they are `Any` or `Union` (or rewrite code to avoid this)
  - **Avoid abstract types** in containers
    - Use concrete types
    - Or, for more flexibility, parametric types
  - Avoid **untyped** containers, never use `[]`, use `Int[]`/`Float[]`
  - Avoid **changing** types, e.g. `x = 1; x / 2` converts int to float
  - Isolate instabilities with **computational kernels**

- Access arrays in **column major** order

- Make use of performance annotations if applicable:
  - `@inbounds` - no bounds checking
  - `@simd` - fused repeated operations 
  - `@fastmath` - fused multiply add operations

- Remember operations have different costs, e.g. `*` is faster than `/`

# References

- https://github.com/carstenbauer/JuliaHLRS22/blob/live/Day2/1_optimizing_serial_performance.ipynb
- https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-abstract-container
- https://docs.julialang.org/en/v1/manual/performance-tips/#kernel-functions
- https://book.sciml.ai/notes/02-Optimizing_Serial_Code/